In [0]:
pip install ccxt


     |████████████████████████████████| 1.5MB 2.7MB/s 
     |████████████████████████████████| 1.2MB 41.6MB/s 
     |████████████████████████████████| 2.3MB 39.4MB/s 
     |████████████████████████████████| 256kB 56.2MB/s 
     |████████████████████████████████| 153kB 53.8MB/s 
     |████████████████████████████████| 235kB 58.7MB/s 
  Created wheel for idna-ssl: filename=idna_ssl-1.1.0-cp36-none-any.whl size=3162 sha256=fc937604282e36f85371de0680892e047333349ad86b24794b3a239a16e6c57c
  Stored in directory: /root/.cache/pip/wheels/d3/00/b3/32d613e19e08a739751dd6bf998cfed277728f8b2127ad4eb7
Successfully built idna-ssl


In [0]:
from google.colab import drive
drive.mount('./gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at ./gdrive


In [0]:
#現在の所持金額の取得
import ccxt
from pprint import pprint
import time
import pandas as pd
import numpy as np

bitflyer = ccxt.bitflyer()
bitflyer.apiKey = 'ScdrTUJ6AkYX1Fwk3u6kWe'
bitflyer.secret = 't81+1ptGH5wdkxITxxJkf21zBy+NUrVaNqaD4971zI4='
total_result = 0
pred_side = "unknown"
comp_pred_number ="nan"
#所持金の確認
def collateral_check():
    global private_collateral
    collateral = bitflyer.private_get_getcollateral()
    private_collateral=collateral['collateral']
#    print("所持金",private_collateral)
    return private_collateral

#send order
def order(order_side,order_lot):
    order = bitflyer.create_order(
    symbol = 'BTC/JPY',
    type='market',
    side=order_side,
    amount=order_lot,
    params = { "product_code" : "FX_BTC_JPY" })
    #print(order_side,order_amount,"lot注文しました")
    #order('SELL','0.03')
    #return side, lot

#POSITION CLOSE
#残ポジションの決済
def position_close():
    positions = bitflyer.private_get_getpositions( params = { "product_code" : "FX_BTC_JPY" })
    #ポジション数
    position_count=len(positions)
    i=0
    position_size=0
    if position_count != 0:
        for i in range(position_count):
        
            if positions[i]['side'] =='BUY':
                position_close = 'SELL'
            else:
                position_close = 'BUY'
        
            position_size=position_size+positions[i]['size']
            position_count= position_size
        
        #決済注文
        order(str(position_close),position_count)
        print("クローズ　　　  :",'{}'.format(position_size),"lot", positions[i]['side'])

#position check
def position_check():
    positions = bitflyer.private_get_getpositions( params = { "product_code" : "FX_BTC_JPY" })
    print("保有ポジション  :",len(positions),"LOT")
    return len(positions)

#get predict infomation
def pred_info():
    global order_time,pred_side,pred_accr, pred_date, pred_number
    df_judge = pd.read_csv('./gdrive/My Drive/Bot_materials/materials/plots/resul.csv')
    order_time = time.time()
    pred_side = df_judge.iat[0,1]
    pred_accr = df_judge.iat[1,1]
    pred_date = df_judge.iat[2,1]
    pred_number = df_judge.iat[3,1]
    return order_time,pred_side,pred_accr, pred_date, pred_number

collateral_check()
#現在の所持金（損益計算用）
start_collateral=private_collateral
print("初期所持金  　　:",private_collateral)
 

while True:
    try:
        print("")
        print("-------------------------------------------------------------------------------")
        #すべてのポジションを確認して決済する
        while float(position_check()) > 0:
            position_close()
            
        #所持金の確認
        collateral_check()
        #取引前の所持金（損益計算用）
        prev_collateral=private_collateral
        print("取引前所持金    :",prev_collateral)
 
        pred_info()
        print("予測更新待ち…")
        while pred_number == comp_pred_number:
            pred_info()
        
        print("予測方向確定待ち…")
        while pred_side == 'unknown':
            #予測結果を取得する
            pred_info()
        

        #注文を出す
        print("10秒後予測      :",pred_side)
        print("予測精度",pred_accr)
        if pred_side == 'UP':
            pred_side = 'BUY'
        else:
            pred_side = 'SELL' 

        order(pred_side,'0.01')
        #一定時間後ポジションを決済する
        time.sleep(10)
        position_close()
        collateral_check()
        recent_collateral=private_collateral
        print("取引後所持金    :",recent_collateral)
 
        #損益計算
        result=recent_collateral-prev_collateral
        print("取引結果        :",result)
        #損益合計
        total_result=recent_collateral - start_collateral
        print("合計損益        :",total_result)
        
        comp_pred_number = pred_number
        print("注文番号        :", comp_pred_number)
    except ccxt.BaseError as e:
        position_close()
        print("error発生")

初期所持金  　　: 9865.0

-------------------------------------------------------------------------------
保有ポジション  : 0 LOT
取引前所持金    : 9865.0
予測更新待ち…
予測方向確定待ち…
10秒後予測      : DOWN
予測精度 0.48315564
クローズ　　　  : 0.01 lot SELL
取引後所持金    : 9839.0
取引結果        : -26.0
合計損益        : -26.0
注文番号        : 11

-------------------------------------------------------------------------------
保有ポジション  : 0 LOT
取引前所持金    : 9839.0
予測更新待ち…
予測方向確定待ち…
10秒後予測      : UP
予測精度 0.6320572
クローズ　　　  : 0.01 lot BUY
取引後所持金    : 9837.0
取引結果        : -2.0
合計損益        : -28.0
注文番号        : 14

-------------------------------------------------------------------------------
保有ポジション  : 0 LOT
取引前所持金    : 9837.0
予測更新待ち…
予測方向確定待ち…
10秒後予測      : DOWN
予測精度 0.5850566
クローズ　　　  : 0.01 lot SELL
取引後所持金    : 9836.0
取引結果        : -1.0
合計損益        : -29.0
注文番号        : 28

-------------------------------------------------------------------------------
保有ポジション  : 0 LOT
取引前所持金    : 9836.0
予測更新待ち…
予測方向確定待ち…


KeyboardInterrupt: ignored